# Combine
Combine the stops if the distance between them equals to zero  
The step is to make sure all stops can get a course change (if same stops are not identified, course change is not available)

In [2]:
import pandas as pd
import numpy as np
import os
import glob

In [3]:
Source = '/Users/zhangwenlan/Downloads/Tongji_RA/数据/Process/Processed/Added2/'
Destion = '/Users/zhangwenlan/Downloads/Tongji_RA/数据/Process/Processed/0Combine/'

In [4]:
os.chdir(Source)
for file in glob.glob("*.csv"):
    df = pd.read_csv(file)      # Read file
    df = df[df['MeanLat'].notna()]

    print(file)
    NewCol = df.columns
    dfNew = pd.DataFrame(columns=NewCol)
    NewStopID = 0
    
    for i, row in df.iterrows():
        if i == 0:
            dfNew.loc[i] = row
            NewStopID += 1
        if i > 0:
            if df.loc[i]['Distance(m)'] <= 0: # 与前一个点相同：合并
                dfNew.at[NewStopID-1, 'EndRow'] = df.loc[i]['EndRow']
                dfNew.at[NewStopID-1, 'EndTime'] = df.loc[i]['EndTime']
                
                if df.loc[i]['TrueStopFixed'] == 1:
                    dfNew.at[NewStopID-1, 'TrueStopFixed'] = df.loc[i]['TrueStopFixed']

            # elif 50 < df.loc[i]['Distance(m)'] < 200000:  # 过大：直接舍弃
            #     dfNew.loc[NewStopID] = row
            #     dfNew.at[NewStopID, 'StopID'] = NewStopID
            #
            #     NewStopID += 1
            
            else:                                           # 不同点
                dfNew.loc[NewStopID] = row
                dfNew.at[NewStopID, 'StopID'] = NewStopID
                NewStopID += 1

    # Duration & Time Delta Calculation
    dfNew['StartTime'] = pd.to_datetime(dfNew['StartTime'], format='%Y/%m/%d')
    dfNew['EndTime'] = pd.to_datetime(dfNew['EndTime'], format='%Y/%m/%d')
    dfNew['Duration(min)'] = (dfNew['EndTime'] - dfNew['StartTime']).astype('timedelta64[m]')
    
    for i, row in dfNew.iterrows():
        if i == 0:
            dfNew.at[i, 'Time Delta'] = 0
        else:
            dfNew.at[i, 'Time Delta'] = pd.Timedelta.total_seconds(dfNew.loc[i]['StartTime'] - dfNew.loc[i-1]['EndTime'])//60
            # df.at[i, 'Time Delta'] = pd.Timedelta.to_timedelta64(df.at[i, 'Time Delta'])        # Unit: Nanosecond

#     dfNew.to_csv(Destion + file, index=False)  # Convert to CSV


FileNotFoundError: [Errno 2] No such file or directory: '/Users/zhangwenlan/Downloads/Tongji_RA/数据/Process/Processed/Added2/'